 ## Data cleaning

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [29]:
!wget -O loan_train.csv https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/loan_train.csv

--2019-07-31 16:25:59--  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/loan_train.csv
Resolving s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)... 67.228.254.193
Connecting to s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)|67.228.254.193|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23101 (23K) [text/csv]
Saving to: ‘loan_train.csv’

100%[======================================>] 23,101      --.-K/s   in 0.002s  

2019-07-31 16:25:59 (11.5 MB/s) - ‘loan_train.csv’ saved [23101/23101]



In [30]:
df = pd.read_csv("loan_train.csv")

df.head(6)

,Unnamed: 0,Unnamed: 0.1,loan_status,Principal,terms,effective_date,due_date,age,education,Gender
0,0,0,PAIDOFF,1000,30,9/8/2016,10/7/2016,45,High School or Below,male
1,2,2,PAIDOFF,1000,30,9/8/2016,10/7/2016,33,Bechalor,female
2,3,3,PAIDOFF,1000,15,9/8/2016,9/22/2016,27,college,male
3,4,4,PAIDOFF,1000,30,9/9/2016,10/8/2016,28,college,female
4,6,6,PAIDOFF,1000,30,9/9/2016,10/8/2016,29,college,male
5,7,7,PAIDOFF,1000,30,9/9/2016,10/8/2016,36,college,male


In [37]:
df['education'].unique().tolist()

['High School or Below', 'Bechalor', 'college', 'Master or Above']

In [75]:
df[["due_date","effective_date"]] = df[["due_date","effective_date"]].astype(str)
df["due_date"] = df["due_date"].str.replace("/","").astype(int)
df["effective_date"] = df["effective_date"].str.replace("/","").astype(int)
df.dtypes

Unnamed: 0         int64
Unnamed: 0.1       int64
loan_status       object
Principal          int64
terms              int64
effective_date     int64
due_date           int64
age                int64
education         object
Gender            object
dtype: object

In [74]:
X = df[["Unnamed: 0","Unnamed: 0.1","Principal","terms","effective_date","due_date","age","education","Gender"]].values
Y = df["loan_status"]

In [43]:
#df["effective_date"] = df["effective_date"].astype(str)
#df["effective_date"] = df["effective _date"].dt.strftime("%m%d%Y").astype(int)

#df["effective _date"] = df["effective_date"].str.replace("/","").astype(int)




## Preprocessing

In [73]:
from sklearn.preprocessing import LabelEncoder
la_en = LabelEncoder()
X[:,7] = la_en.fit_transform(X[:,7])


In [72]:
X[:,8] = la_en.fit_transform(X[:,8])

In [35]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size = 0.2 ,random_state= 4)
print("Train size",X_train.shape)
print("Test size",X_test.shape, Y_test.shape)

Train size (276, 9)
Test size (70, 9) (70,)


# 1.KNN implement

In [36]:
from sklearn.neighbors import KNeighborsClassifier
k = 6
neigh = KNeighborsClassifier(n_neighbors=k).fit(X_train,Y_train)
neigh

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=6, p=2,
           weights='uniform')

### i)prediction

In [37]:
y_hat = neigh.predict(X_test)
y_hat[0:5]

array(['PAIDOFF', 'PAIDOFF', 'PAIDOFF', 'PAIDOFF', 'PAIDOFF'],
      dtype=object)

### ii ) Accuracy calculation

In [38]:
## accuracy_metrics
from sklearn import metrics
print("train set accuracy",metrics.accuracy_score(Y_train,neigh.predict(X_train)))
print("test set accuracy",metrics.accuracy_score(Y_test,y_hat))

## f1_score
from sklearn.metrics import f1_score
f1_score(Y_test,y_hat,average= "weighted")

## jaccard_index
from sklearn.metrics import jaccard_similarity_score
jaccard_similarity_score(Y_test,y_hat)





train set accuracy 0.9746376811594203
test set accuracy 0.9714285714285714


0.9714285714285714

#  2.SVM Implementation:

In [39]:
from sklearn import svm
clf = svm.SVC(kernel = 'rbf')
clf.fit(X_train,Y_train)


/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

### i) prediction

In [59]:
y_hat1 = clf.predict(X_test)

### ii) Accuracy calculation

In [61]:
## accuracy metrics
from sklearn import metrics
print("train set accuracy",metrics.accuracy_score(Y_train,clf.predict(X_train)))
print("test set accuracy",metrics.accuracy_score(Y_test,y_hat))

# f1_score

from sklearn.metrics import f1_score
f1_score(Y_test,y_hat1,average= 'weighted')

# jaccard index

from sklearn.metrics import jaccard_similarity_score
jaccard_similarity_score(Y_test,y_hat1)

train set accuracy 1.0
test set accuracy 0.9714285714285714


0.8

# 3.Decision Tree implementation

In [55]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(criterion ="entropy",max_depth = 6)
dt.fit(X_train,Y_train)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=6,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

### i) Prediction

In [56]:
pt = dt.predict(X_test)

print(pt[0:5])
print(Y_test[0:5])

['PAIDOFF' 'PAIDOFF' 'PAIDOFF' 'PAIDOFF' 'PAIDOFF']
63     PAIDOFF
34     PAIDOFF
186    PAIDOFF
146    PAIDOFF
130    PAIDOFF
Name: loan_status, dtype: object


### ii) Accuracy Calculation

In [57]:
from sklearn.metrics import accuracy_score
print("Decision Tree Accuracy", accuracy_score(Y_test,pt))

Decision Tree Accuracy 1.0


# 4.Logistic Regression Implementation

In [66]:
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression().fit(X_train,Y_train)
LR

/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

### i) Prediction

In [67]:
y_hat2 = LR.predict(X_test)
y_hat2

array(['PAIDOFF', 'PAIDOFF', 'PAIDOFF', 'PAIDOFF', 'PAIDOFF', 'PAIDOFF',
       'PAIDOFF', 'COLLECTION', 'PAIDOFF', 'PAIDOFF', 'PAIDOFF',
       'PAIDOFF', 'PAIDOFF', 'PAIDOFF', 'PAIDOFF', 'PAIDOFF', 'PAIDOFF',
       'PAIDOFF', 'PAIDOFF', 'PAIDOFF', 'PAIDOFF', 'PAIDOFF',
       'COLLECTION', 'PAIDOFF', 'PAIDOFF', 'PAIDOFF', 'PAIDOFF',
       'PAIDOFF', 'PAIDOFF', 'PAIDOFF', 'PAIDOFF', 'COLLECTION',
       'PAIDOFF', 'PAIDOFF', 'PAIDOFF', 'PAIDOFF', 'PAIDOFF', 'PAIDOFF',
       'PAIDOFF', 'PAIDOFF', 'COLLECTION', 'PAIDOFF', 'PAIDOFF',
       'PAIDOFF', 'PAIDOFF', 'PAIDOFF', 'PAIDOFF', 'PAIDOFF', 'PAIDOFF',
       'PAIDOFF', 'PAIDOFF', 'PAIDOFF', 'PAIDOFF', 'PAIDOFF', 'PAIDOFF',
       'PAIDOFF', 'PAIDOFF', 'PAIDOFF', 'PAIDOFF', 'PAIDOFF', 'PAIDOFF',
       'PAIDOFF', 'PAIDOFF', 'PAIDOFF', 'PAIDOFF', 'PAIDOFF', 'PAIDOFF',
       'PAIDOFF', 'PAIDOFF', 'PAIDOFF'], dtype=object)

### ii)Accuracy Calculation

In [68]:
## jaccard index
from sklearn.metrics import jaccard_similarity_score
jaccard_similarity_score(Y_test,y_hat2)

## Accuracy metrics

from sklearn import metrics
print("Train set Accuracy",metrics.accuracy_score(Y_train,LR.predict(X_train)))
print("Test set Accuracy",metrics.accuracy_score(Y_test,y_hat2))

#print(Y_test[0:10])
#print(y_hat2[0:10])

Train set Accuracy 0.7644927536231884
Test set Accuracy 0.7571428571428571
